In [1]:
import speech_recognition as sr
import pyttsx3 
import json
import numpy as np 
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors
import colorama
colorama.init
from colorama import Fore, Style, Back
import random
import pickle

In [2]:
with open('intents.json') as file:
    data = json.load(file)

In [3]:
df = pd.read_csv('books.csv', error_bad_lines = False)

C:\Users\DELL\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


In [4]:
# converting average rating column into categorical column
def num_into_obj(x):
    if x>=0 and x<=1:
        return 'between 0 and 1'
    elif x>1 and x<=2:
        return 'between 1 and 2'
    elif x>2 and x<=3:
        return 'between 2 and 3'
    elif x>3 and x<=4:
        return 'between 3 and 4'
    else:
        return 'between 4 and 5'
    
df['rating_obj'] = df['average_rating'].apply(num_into_obj)

In [5]:
# Let's encode the categorical column
rating_df = pd.get_dummies(df['rating_obj'])
#rating_df.head()

In [6]:
# Let's encode the language code column as well
language_df = pd.get_dummies(df['language_code'])
#language_df.head()

In [7]:
# Let's concat both the data frames and set the title column as the index 
features = pd.concat([rating_df, language_df, df['average_rating'], df['ratings_count'],df['title']], axis=1)
features.set_index('title', inplace=True)
#features.head()

In [8]:
min_max_scaler = MinMaxScaler()
features_scaled = min_max_scaler.fit_transform(features)

In [9]:
model = neighbors.NearestNeighbors(n_neighbors=6, algorithm='ball_tree', metric='euclidean')
model.fit(features_scaled)
dist, idlist = model.kneighbors(features_scaled)

In [10]:
# Initialize the recognizer 
r = sr.Recognizer() 

In [11]:
# Function to convert text to
# speech
def SpeakText(command):
      
    # Initialize the engine
    engine = pyttsx3.init()
    engine.say(command) 
    engine.runAndWait()



In [ ]:
# Loop infinitely for user to
# speak
model = keras.models.load_model('voice_assistant')
with open('tokenizer.pickle','rb') as handle:
    tokenizer = pickle.load(handle)
with open('lbl_encoder.pickle','rb') as ecn:
    lbl_encoder = pickle.load(ecn)
max_len = 20  
while(1):    
      
    # Exception handling to handle
    # exceptions at the runtime
    try:
          
        # use the microphone as source for input.
        with sr.Microphone() as source2:
              
            # wait for a second to let the recognizer
            # adjust the energy threshold based on
            # the surrounding noise level 
            r.adjust_for_ambient_noise(source2, duration=0.2)
              
            #listens for the user's input 
            audio2 = r.listen(source2)
              
            # Using ggogle to recognize audio
            MyText = r.recognize_google(audio2)
            MyText = MyText.lower()
  

        
            #print(Fore.LIGHTBLUE_EX + "User: "+ Style.RESET_ALL,end="")
            #inp = input()
            if MyText.lower() == "bye":
                break
            if MyText.lower() == "book search":
                print("Tell book name you like")
                SpeakText("Tell book name you like")
                while(1):
                    try:
                        with sr.Microphone() as source2:
                            r.adjust_for_ambient_noise(source2, duration=0.2)
                            audio2 = r.listen(source2)
                            MyBook = r.recognize_google(audio2)
                            MyBooks = MyBook.lower()
                            book=MyBooks.title()
                            print(book)
                                 
                            def BookRecommender(book):
                                book_list_name = []
                                book_id = df[df['title'] == book].index
                                book_id = book_id[0]
                                for newid in idlist[book_id]:
                                    book_list_name.append(df.loc[newid].title)
                                    SpeakText(df.loc[newid].title)
                                    print(df.loc[newid].title)
                                return
                            if MyText.lower() == "thank you":
                                break
                            #print(Fore.GREEN + "NUBI: "+"I would prefer that you read.")
                            #SpeakText("I would prefer that you read")
                            BookRecommender(book)    
                            
                    except sr.RequestError as e:
                        print("Could not request results; {0}".format(e))
                        
            result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([MyText]),
                                                                           truncating='post',maxlen=max_len))
        
            tag = lbl_encoder.inverse_transform([np.argmax(result)])
            for i in data['intents']:
                if i['tag']==tag:
                    print(Fore.GREEN + "NUBI: "+ Style.RESET_ALL,np.random.choice(i['responses']))
                    #print("Did you say "+np.random.choice(i['responses']))
                    SpeakText(np.random.choice(i['responses']))     
            
    except sr.RequestError as e:
        print("Could not request results; {0}".format(e))
          
    except sr.UnknownValueError:
        print( "NUBI:I can't hear you")
        SpeakText("I can't hear you")